In [1]:
from slippi import Game
from slippi.parse import parse
from slippi.parse import ParseEvent
import pandas as pd
from os import listdir
from os.path import isfile, join
import tqdm
import time
from copy import deepcopy

In [2]:
def parse_frame_at_port(game, frame, port):
    return [game.frames[frame].ports[port].leader.pre, game.frames[frame].ports[port].leader.post]

In [3]:
def read_game(file_name):
    try:
        return Game(f'../Replays/{file_name}')
    except:
        print(f'Failed to parse {file_name}')

In [4]:
def add_replay(file_name, d, k):
    game = read_game(file_name)
    if game is None:
            print(f'{file_name} is None')
    else: 
        #get uses ports
        pls = [i for i, p in enumerate(game.start.players) if not(p is None)]
        player_info = game.start.players
        char_1 = player_info[pls[0]].character.name
        char_2 = player_info[pls[1]].character.name
        head = [file_name, char_1, char_2]
        #add each frame
        for f in range(len(game.frames)):
            d[k] = head + [f] + parse_frame_at_port(game, f, pls[0]) + parse_frame_at_port(game, f, pls[1])
            k += 1
    return d, k

In [5]:
def replays_to_df(replay_names):
    data = dict()
    k = 0
    for replay in replay_names:
        data, k = add_replay(replay, data, k)
    df = pd.DataFrame.from_dict(data, columns=['Game_ID','CHAR_P1', 'CHAR_P2','Frame', 'Pre_P1', 'Post_P1', 'Pre_P2', 'Post_P2'], orient='index')
    return df

In [6]:
def get_states(df):
    for i in [1,2]:
        post = df[f'Post_P{i}']
        df[f'S_airborne_P{i}'] = post.apply(lambda x : x.airborne).values
        df[f'S_damage_P{i}'] = post.apply(lambda x : x.damage).values
        df[f'S_direction_P{i}'] = post.apply(lambda x : x.direction).values
        df[f'S_hit_stun_P{i}'] = post.apply(lambda x : x.hit_stun).values
        df[f'S_position_x_P{i}'] = post.apply(lambda x : x.position.x).values
        df[f'S_position_y_P{i}'] = post.apply(lambda x : x.position.y).values
        df[f'S_shield_P{i}'] = post.apply(lambda x : x.shield).values
        df[f'S_state_P{i}'] = post.apply(lambda x : x.state).values
        df[f'S_state_age_P{i}'] = post.apply(lambda x : x.state_age).values
        df[f'S_stocks_P{i}'] = post.apply(lambda x : x.stocks).values
    
    df = df.drop(['Post_P1', 'Post_P2'], axis = 1)
    return df

In [7]:
def get_buttons(df):
    for i in [1,2]:
        pre = df[f'Pre_P{i}']
        df[f'B_damage_P{i}'] = pre.apply(lambda x : x.damage).values
        df[f'B_direction_P{i}'] = pre.apply(lambda x : x.damage).values
        df[f'B_joystick_x_P{i}'] = pre.apply(lambda x : x.joystick.x).values
        df[f'B_joystick_y_P{i}'] = pre.apply(lambda x : x.joystick.y).values
        df[f'B_position_x_P{i}'] = pre.apply(lambda x : x.position.x).values
        df[f'B_position_y_P{i}'] = pre.apply(lambda x : x.position.y).values
        df[f'B_cstick_x_P{i}'] = pre.apply(lambda x : x.cstick.x).values
        df[f'B_cstick_y_P{i}'] = pre.apply(lambda x : x.cstick.y).values
        df[f'B_state_P{i}'] = pre.apply(lambda x : x.state).values
        df[f'B_raw_analog_P{i}'] = pre.apply(lambda x : x.raw_analog_x).values
        df[f'B_buttons_physical_P{i}'] = pre.apply(lambda x : x.buttons.physical.value).values
        df[f'B_buttons_physical_P{i}'] = pre.apply(lambda x : x.buttons.logical.value).values
        df[f'B_triggers_physical_l_P{i}'] = pre.apply(lambda x : x.triggers.physical.l).values
        df[f'B_triggers_physical_r_P{i}'] = pre.apply(lambda x : x.triggers.physical.r).values
        df[f'B_triggers_logical_P{i}'] = pre.apply(lambda x : x.triggers.logical).values
    
    df = df.drop(['Pre_P1', 'Pre_P2'], axis = 1)
    return df

In [23]:
def df_column_switch(df, column1, column2):
    i = list(df.columns)
    a, b = i.index(column1), i.index(column2)
    i[b], i[a] = i[a], i[b]
    df = df[i]
    return df

In [27]:
%%time
files = listdir('../Replays/')
df = replays_to_df(files[0:5])

Wall time: 13.6 s


In [19]:
offset = 7

In [29]:
df = get_states(df)
df = get_buttons(df)

In [46]:
mapping = {'B_damage_P2': 'B_damage_P1',
 'B_direction_P2': 'B_direction_P1',
 'B_joystick_x_P2': 'B_joystick_x_P1',
 'B_joystick_y_P2': 'B_joystick_y_P1',
 'B_position_x_P2': 'B_position_x_P1',
 'B_position_y_P2': 'B_position_y_P1',
 'B_cstick_x_P2': 'B_cstick_x_P1',
 'B_cstick_y_P2': 'B_cstick_y_P1',
 'B_state_P2': 'B_state_P1',
 'B_raw_analog_P2': 'B_raw_analog_P1',
 'B_buttons_physical_P2': 'B_buttons_physical_P1',
 'B_triggers_physical_l_P2': 'B_triggers_physical_l_P1',
 'B_triggers_physical_r_P2': 'B_triggers_physical_r_P1',
 'B_triggers_logical_P2': 'B_triggers_logical_P1'}

In [54]:
X_cols = ['Game_ID', 'CHAR_P1', 'CHAR_P2', 'Frame', 'S_airborne_P1',
       'S_damage_P1', 'S_direction_P1', 'S_hit_stun_P1', 'S_position_x_P1',
       'S_position_y_P1', 'S_shield_P1', 'S_state_P1', 'S_state_age_P1',
       'S_stocks_P1', 'S_airborne_P2', 'S_damage_P2', 'S_direction_P2',
       'S_hit_stun_P2', 'S_position_x_P2', 'S_position_y_P2', 'S_shield_P2',
       'S_state_P2', 'S_state_age_P2', 'S_stocks_P2']

x_cols_P1 = ['B_damage_P1',
       'B_direction_P1', 'B_joystick_x_P1', 'B_joystick_y_P1',
       'B_position_x_P1', 'B_position_y_P1', 'B_cstick_x_P1', 'B_cstick_y_P1',
       'B_state_P1', 'B_raw_analog_P1', 'B_buttons_physical_P1',
       'B_triggers_physical_l_P1', 'B_triggers_physical_r_P1',
       'B_triggers_logical_P1']
          
x_cols_P2 =['B_damage_P2', 'B_direction_P2',
       'B_joystick_x_P2', 'B_joystick_y_P2', 'B_position_x_P2',
       'B_position_y_P2', 'B_cstick_x_P2', 'B_cstick_y_P2', 'B_state_P2',
       'B_raw_analog_P2', 'B_buttons_physical_P2', 'B_triggers_physical_l_P2',
       'B_triggers_physical_r_P2', 'B_triggers_logical_P2']

y_cols = ['Game_ID','Frame', 'S_airborne_P1',
       'S_damage_P1', 'S_direction_P1', 'S_hit_stun_P1', 'S_position_x_P1',
       'S_position_y_P1', 'S_shield_P1', 'S_state_P1', 'S_state_age_P1',
       'S_stocks_P1', 'S_airborne_P2', 'S_damage_P2', 'S_direction_P2',
       'S_hit_stun_P2', 'S_position_x_P2', 'S_position_y_P2', 'S_shield_P2',
       'S_state_P2', 'S_state_age_P2', 'S_stocks_P2']

In [55]:
df_y_target = df[y_cols]
df_y_target['Frame'] = df_X_target['Frame'].apply(lambda x : x - offset)

C:\Users\steph\AppData\Local\Temp/ipykernel_16024/1848564158.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_y_target['Frame'] = df_X_target['Frame'].apply(lambda x : x - offset)


In [40]:
df_X_P1 = df[X_cols + x_cols_P1]
df_X_P2 = df[X_cols + x_cols_P2]

In [41]:
df_X_P2 = df_column_switch(df_X_P2, 'CHAR_P1', 'CHAR_P2')

In [45]:
mapping = {}
for c_name in y_cols_P2:
    mapping[c_name] = c_name[:-1] + '1'
mapping

{'B_damage_P2': 'B_damage_P1',
 'B_direction_P2': 'B_direction_P1',
 'B_joystick_x_P2': 'B_joystick_x_P1',
 'B_joystick_y_P2': 'B_joystick_y_P1',
 'B_position_x_P2': 'B_position_x_P1',
 'B_position_y_P2': 'B_position_y_P1',
 'B_cstick_x_P2': 'B_cstick_x_P1',
 'B_cstick_y_P2': 'B_cstick_y_P1',
 'B_state_P2': 'B_state_P1',
 'B_raw_analog_P2': 'B_raw_analog_P1',
 'B_buttons_physical_P2': 'B_buttons_physical_P1',
 'B_triggers_physical_l_P2': 'B_triggers_physical_l_P1',
 'B_triggers_physical_r_P2': 'B_triggers_physical_r_P1',
 'B_triggers_logical_P2': 'B_triggers_logical_P1'}

In [52]:
df_X_P2.rename(columns=mapping, inplace=True)

In [56]:
out1 = pd.merge(df_X_P1, df_y_target, on=['Game_ID', 'Frame'])
out2 = pd.merge(df_X_P2, df_y_target, on=['Game_ID', 'Frame'])

In [60]:
df_y_target

,Game_ID,Frame,S_airborne_P1,S_damage_P1,S_direction_P1,S_hit_stun_P1,S_position_x_P1,S_position_y_P1,S_shield_P1,S_state_P1,...,S_airborne_P2,S_damage_P2,S_direction_P2,S_hit_stun_P2,S_position_x_P2,S_position_y_P2,S_shield_P2,S_state_P2,S_state_age_P2,S_stocks_P2
0,001AE992EB29_20210714T231243.slp,-14,True,0.0,-1,5.605194e-45,-39.999996,31.999992,60.0,322,...,True,0.000000,1,2.662467e-44,-70.00,7.0,60.0,322,-1.0,4
1,001AE992EB29_20210714T231243.slp,-13,True,0.0,-1,4.203895e-45,-39.999996,31.999992,60.0,322,...,True,0.000000,1,2.522337e-44,-70.00,7.0,60.0,322,-1.0,4
2,001AE992EB29_20210714T231243.slp,-12,True,0.0,-1,2.802597e-45,-39.999996,31.999992,60.0,322,...,True,0.000000,1,2.382207e-44,-70.00,7.0,60.0,322,-1.0,4
3,001AE992EB29_20210714T231243.slp,-11,True,0.0,-1,1.401298e-45,-39.999996,31.999992,60.0,322,...,True,0.000000,1,2.242078e-44,-70.00,7.0,60.0,322,-1.0,4
4,001AE992EB29_20210714T231243.slp,-10,True,0.0,-1,0.000000e+00,-39.999996,31.999992,60.0,322,...,True,0.000000,1,2.101948e-44,-70.00,7.0,60.0,322,-1.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51763,1008_Falcon_vs_Fox_(BF).slp,6820,True,10.0,1,2.298830e-41,73.982941,-37.595432,60.0,354,...,True,28.880001,-1,7.006492e-45,70.32,-14.4,60.0,253,26.0,2
51764,1008_Falcon_vs_Fox_(BF).slp,6821,True,10.0,1,3.643376e-44,74.357758,-39.120373,60.0,35,...,True,28.880001,-1,7.006492e-45,70.32,-14.4,60.0,253,27.0,2
51765,1008_Falcon_vs_Fox_(BF).slp,6822,True,10.0,1,3.783506e-44,74.722572,-40.775314,60.0,35,...,True,28.880001,-1,7.006492e-45,70.32,-14.4,60.0,253,28.0,2
51766,1008_Falcon_vs_Fox_(BF).slp,6823,True,10.0,1,3.783506e-44,75.077393,-42.560253,60.0,35,...,True,28.880001,-1,7.006492e-45,70.32,-14.4,60.0,253,29.0,2


In [57]:
out = pd.concat([out1,out2],ignore_index=True)

In [59]:
out.columns

Index(['Game_ID', 'CHAR_P1', 'CHAR_P2', 'Frame', 'S_airborne_P1_x',
       'S_damage_P1_x', 'S_direction_P1_x', 'S_hit_stun_P1_x',
       'S_position_x_P1_x', 'S_position_y_P1_x', 'S_shield_P1_x',
       'S_state_P1_x', 'S_state_age_P1_x', 'S_stocks_P1_x', 'S_airborne_P2_x',
       'S_damage_P2_x', 'S_direction_P2_x', 'S_hit_stun_P2_x',
       'S_position_x_P2_x', 'S_position_y_P2_x', 'S_shield_P2_x',
       'S_state_P2_x', 'S_state_age_P2_x', 'S_stocks_P2_x', 'B_damage_P1',
       'B_direction_P1', 'B_joystick_x_P1', 'B_joystick_y_P1',
       'B_position_x_P1', 'B_position_y_P1', 'B_cstick_x_P1', 'B_cstick_y_P1',
       'B_state_P1', 'B_raw_analog_P1', 'B_buttons_physical_P1',
       'B_triggers_physical_l_P1', 'B_triggers_physical_r_P1',
       'B_triggers_logical_P1', 'S_airborne_P1_y', 'S_damage_P1_y',
       'S_direction_P1_y', 'S_hit_stun_P1_y', 'S_position_x_P1_y',
       'S_position_y_P1_y', 'S_shield_P1_y', 'S_state_P1_y',
       'S_state_age_P1_y', 'S_stocks_P1_y', 'S_airborn

In [ ]:

#

# df_X2 = df_column_switch(df_X, 'CHAR_P1', 'CHAR_P2')

# df_y_P2.columns=['Game_ID','Frame','B_damage_P1',
#        'B_direction_P1', 'B_joystick_x_P1', 'B_joystick_y_P1',
#        'B_position_x_P1', 'B_position_y_P1', 'B_cstick_x_P1', 'B_cstick_y_P1',
#        'B_state_P1', 'B_raw_analog_P1', 'B_buttons_physical_P1',
#        'B_triggers_physical_l_P1', 'B_triggers_physical_r_P1',
#        'B_triggers_logical_P1']

# out2 = pd.merge(df_X2, df_y_P2, on=['Game_ID', 'Frame'])

# out = pd.concat([out1,out2],ignore_index=True)